### Runnable 클래스
Runnable 클래스는 LangChain의 핵심 실행 단위(Execution Unit) 로,
LLM 호출·체인 구성·데이터 변환 등 모든 실행 가능한 구성요소의 공통 인터페이스를 제공한다.
Runnable은 기본적으로 Runnable 프로토콜을 구현한 객체들이고,서로 파이프( | )로 연결하거나 병렬, 분기 등으로 조합 가능하다

In [1]:
from dotenv import load_dotenv
import os

# .env 파일 불러오기
load_dotenv("C:/env/.env")

True

### RunnableLambda

In [2]:
from langchain_core.runnables import RunnableLambda

# 1) 문자열을 대문자로 변환하는 RunnableLambda
to_upper = RunnableLambda(lambda x:x.upper())

# 2) 문자열 길이를 계산하는 RunnableLambda
count_chars = RunnableLambda(lambda x: f"문자 개수: {len(x)}")

# 3) 두 Runnable을 파이프(|)로 연결
pipeline = to_upper | count_chars

# 실행
result1 = to_upper.invoke("hello langchain")
result2 = pipeline.invoke("hello langchain")

print("단일 RunnableLambda 결과:", result1)
print("파이프라인 연결 결과:", result2)

단일 RunnableLambda 결과: HELLO LANGCHAIN
파이프라인 연결 결과: 문자 개수: 15


### RunnableMap
- 여러 Runnable을 병렬로 실행하고 결과를 dict 형태로 묶어주는 클래스

In [6]:
from langchain_core.runnables import RunnableLambda, RunnableMap

# 1) Runnable 정의
to_upper = RunnableLambda(lambda x:x.upper())    # 대문자 변환
count_chars = RunnableLambda(lambda x: len(x))   # 문자 수 반환
reverse_text = RunnableLambda(lambda x: x[::-1]) # 문자열 뒤집기

# 2) RunnableMap으로 묶기
parallel_map = RunnableMap({
    "대문자": to_upper,
    "길이": count_chars,
    "뒤집기": reverse_text
})

# 3) 실행
result = parallel_map.invoke("RunnableMap Example")
print(result)

{'대문자': 'RUNNABLEMAP EXAMPLE', '길이': 19, '뒤집기': 'elpmaxE paMelbannuR'}


### RunnableSequence
- 순차적으로(직렬) 연결해서 파이프라인을 만드는 클래스

In [8]:
from langchain_core.runnables import RunnableLambda, RunnableSequence

# 1) 1단계: 문자열을 대문자로 변환
to_upper = RunnableLambda(lambda x:x.upper())

# 2) 2단계: 문자열 길이를 계산
count_chars = RunnableLambda(lambda x: f"문자 개수: {len(x)}")

# 3) RunnableSequence로 순차 실행 파이프라인 생성
sequence = RunnableSequence(first=to_upper,middle=[],last=count_chars)

# 실행
result = sequence.invoke("Hello RunnableSequence")

# print(to_upper.invoke("Hello RunnableSequence"))
print("RunnableSequence 실행 결과:", result)

RunnableSequence 실행 결과: 문자 개수: 22


In [10]:
 # to_upper.invoke("Hello RunnableSequence")

### RunnableBranch
- 조건 분기(conditional routing) 를 구현할 때 사용하는 Runnable

In [11]:
from langchain_core.runnables import RunnableLambda, RunnableBranch

# 1) 조건별 Runnable 정의
greet_case = RunnableLambda(lambda x: "인사 감지: 반가워요!")
long_text_case = RunnableLambda(lambda x: f"긴 문장 ({len(x)} 글자) 이네요.")
default_case = RunnableLambda(lambda x: f"일반 입력: {x}")

# 2) RunnableBranch 정의
# 마지막 인자는 조건 없는 default Runnable
branch = RunnableBranch(
    (lambda x: "hello" in x.lower(), greet_case),
    (lambda x: len(x) > 15, long_text_case),
    default_case  # <- 조건 없이 실행될 Runnable
)

# 3) 실행
print(branch.invoke("hello world"))                # hello 포함
print(branch.invoke("이 문장은 길이가 조금 길어요")) # 길이 > 15
print(branch.invoke("짧음"))                       # 기본 케이스

인사 감지: 반가워요!
긴 문장 (16 글자) 이네요.
일반 입력: 짧음


### RunnableParallel 
- 비동기적(asynchronous)으로 여러 호출을 동시에 보내고 결과를 모으는 방식. 서로 다른 서버에서 동시에 처리

In [12]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

# 1) Runnable 정의
to_upper = RunnableLambda(lambda x: x.upper())
count_chars = RunnableLambda(lambda x: len(x))
reverse_text = RunnableLambda(lambda x: x[::-1])

# 2) RunnableParallel로 병렬 실행 정의
parallel = RunnableParallel(
    upper=to_upper,
    length=count_chars,
    reversed=reverse_text
)

# 3) 실행
result = parallel.invoke("Hello RunnableParallel!")

print(result)

{'upper': 'HELLO RUNNABLEPARALLEL!', 'length': 23, 'reversed': '!lellaraPelbannuR olleH'}


### RunnablePassthrough
- 입력을 아무 변경 없이 그대로 반환하는 Runnable

In [18]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# 1) 입력 그대로 반환하는 RunnablePassthrough
passthrough = RunnablePassthrough()

# 2) 입력 문자열을 대문자로 변환하는 RunnableLambda
to_upper = RunnableLambda(lambda x: x.upper())

# 3) 파이프라인: (입력 그대로) -> (대문자로 변환)
pipeline = passthrough | to_upper

# 실행
result1 = passthrough.invoke("Hello Runnable World!")   # 입력 그대로 반환
result2 = pipeline.invoke("Hello Runnable World!")      # 대문자로 변환 결과

print("Passthrough 결과:", result1)
print("Passthrough + Lambda 파이프라인 결과:", result2)

Passthrough 결과: Hello Runnable World!
Passthrough + Lambda 파이프라인 결과: HELLO RUNNABLE WORLD!


### RunnableEach
- 리스트 입력을 받아 각 요소를 같은 Runnable에 적용

In [31]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.base import RunnableEach  # 최신 import 경로

# 1) 문자열을 대문자로 변환하는 Runnable 정의
to_upper = RunnableLambda(lambda x: x.upper())

# 2) RunnableEach로 리스트의 각 요소에 적용 (bound= 필수!)
each_upper = RunnableEach(bound=to_upper)

# 3) 실행 : 리스트 입력을 받아 각 요소를 대문자로 변환
result = each_upper.invoke(["hello", "runnable", "each"])
print(result)

['HELLO', 'RUNNABLE', 'EACH']


In [33]:
# def my_upper(x):
#     return x.upper()
    
# result = map(my_upper,['a','b','c','d','e','1'])
# print(list(result))
# next(result)

### RunnableRetry/RunnableWithFallbacks/RunnableBinding
- RunnableRetry : 불안정한 Runnable을 감싸서 자동 재시도 기능을 추가
네트워크 지연, 일시적 API 오류, 모델 응답 실패 같은 상황에서 유용
- RunnableWithFallbacks : 메인 Runnable이 최종 실패했을 때 대체 Runnable 실행
(예: LLM 응답 실패 → 간단한 기본 응답 반환)
- RunnableBinding :Runnable에 옵션이나 파라미터를 미리 바인딩
매번 .invoke() 호출 시 옵션을 넘길 필요 없음

In [57]:
# 실패할 수도 있는 작업 → 재시도 → 대체(fallback) → 후처리(prefix)”
# 로 이어지는 내결함성(Fault-tolerant) 파이프라인을 구성한 예제
import random
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.retry import RunnableRetry
from langchain_core.runnables.fallbacks import RunnableWithFallbacks
from langchain_core.runnables.base import RunnableBinding

# 1) 실패 확률이 있는 Runnable
unstable = RunnableLambda(lambda x: x if random.random() > 0.5 else (_ for _ in ()).throw(Exception("랜덤 실패 발생")))

# 2) 재시도 Runnable
# unstable을 실행 → 실패 시 다시 시도,3번 모두 실패하면 예외를 그대로 상위로 전달
retry_runnable = RunnableRetry(bound=unstable,max_attempts=3)

# 3) fallback Runnable
# retry_runnable 시도 (최대 3회 재시도),그래도 실패하면 → fallback 실행
fallback = RunnableLambda(lambda x: f" 원래 작업 실패 → fallback 결과: {x}")
with_fallback = RunnableWithFallbacks(runnable=retry_runnable, fallbacks=[fallback])

# 4) prefix 추가 Runnable (Binding으로 옵션 미리 고정)
prefixer = RunnableLambda(lambda x, prefix="결과: ": f"{prefix}{x}")
bound_prefixer = RunnableBinding(bound=prefixer,kwargs={"prefix":"최종 출력 -->"})

# 5) 파이프라인으로 연결
pipeline = with_fallback | bound_prefixer

# 6) 실행
print(pipeline.invoke("테스트 입력"))

최종 출력 -->테스트 입력


In [63]:
unstable.invoke("unstable")  # random.random() 이  0.5 보다 작거나 같으면 Exception("랜덤 실패 발생") 이 실행

'unstable'

In [84]:
retry_runnable.invoke('retry') # unstable 을 3회 시도 , 성공하면 'retry'를 출력,오류시 Exception("랜덤 실패 발생") 이 실행

'retry'

In [42]:
# (_ for _ in ()) → “빈 제너레이터(generator)”를 만든다. (즉, 아무것도 산출하지 않는 반복자)
# .throw(Exception("랜덤 실패 발생"))→ 이 제너레이터에게 즉시 예외를 던진다(raise).
# lambda x: x if random.random() > 0.5 else (_ for _ in ()).throw(Exception("랜덤 실패 발생"))

<function __main__.<lambda>(x)>

In [85]:
# r = random.random()
# if r > 0.5:
#     print('성공:',r)
# else:
#     print('오류:',r)

In [86]:
def my_gen():
    try:
        yield 1
        yield 2
    except Exception as e:
        print("예외 발생:", e)
        yield "예외 처리 후 계속 실행"
    yield 3

g = my_gen()
print(next(g))        # 1
print(g.throw(Exception, "테스트 예외"))  # 예외를 던짐
print(next(g))        # 3
# throw()는 파이썬 제너레이터(generator) 객체에서 사용되는 메서드로,
# 일반적인 함수 호출이 아니라 제너레이터 내부로 예외를 던지는 기능을 한다.

1
예외 발생: 테스트 예외
예외 처리 후 계속 실행
3


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_12948\674312179.py:12: DeprecationWarning: the (type, exc, tb) signature of throw() is deprecated, use the single-arg signature instead.
  print(g.throw(Exception, "테스트 예외"))  # 예외를 던짐


### RunnableWithMessageHistory

In [87]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

# 1) LLM + 프롬프트 (대화 기록 자리 포함)
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder("history"),
    ("human", "{input}")
])
model = ChatOpenAI(model="gpt-4o-mini")
chain = prompt | model

# 2) 세션별 대화 기록 저장 함수
store = {}
def get_history(_):
    store.setdefault("chat-1", InMemoryChatMessageHistory())
    return store["chat-1"]

# 3) RunnableWithMessageHistory로 감싸기
with_history = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 4) 실행
config = {"configurable": {"session_id": "chat-1"}}

print(with_history.invoke({"input": "안녕? 나는 홍길동이야!"}, config=config).content)
print(with_history.invoke({"input": "내 이름이 뭐지?"}, config=config).content)


안녕, 홍길동! 만나서 반가워. 어떻게 도와줄까요?
당신의 이름은 홍길동입니다! 어떤 이야기를 나누고 싶으신가요?


### 다양한 Runnable 연동

In [90]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableMap

# 1) Runnable 정의
# 입력 그대로 전달
passthrough = RunnablePassthrough()

# 대문자로 변환
to_upper = RunnableLambda(lambda x: x.upper())

# 글자 수 계산
count_chars = RunnableLambda(lambda x :len(x))

# 2) 병렬 실행 (Map): 여러 Runnable을 동시에 실행하고 결과를 dict로 반환
parallel = RunnableMap({
    "original": passthrough,  # 입력 그대로
    "upper": to_upper,        # 대문자로 변환
    "length": count_chars     # 글자 수 계산
})

result = parallel.invoke("Hello Runnable World!")
print(result)
print("-"*100)

# 3) 파이프라인 구성
# (입력 그대로) -> (병렬 실행) -> (대문자 결과만 다시 가공)
pipeline = passthrough | parallel | RunnableLambda(
    lambda result: {
        "원본": result["original"],
        "대문자": result["upper"],
        "글자수": result["length"],
        "대문자+길이": f'{result["upper"]} (총 {result["length"]} 글자)'
    }
)

# 실행
result = pipeline.invoke("Hello Runnable World!")
print(result)

{'original': 'Hello Runnable World!', 'upper': 'HELLO RUNNABLE WORLD!', 'length': 21}
----------------------------------------------------------------------------------------------------
{'원본': 'Hello Runnable World!', '대문자': 'HELLO RUNNABLE WORLD!', '글자수': 21, '대문자+길이': 'HELLO RUNNABLE WORLD! (총 21 글자)'}
